In [1]:
import numpy as np

In [2]:
asteroids = """.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##"""

# asteroids = """.#..#..###
# ####.###.#
# ....###.#.
# ..###.##.#
# ##.##.#.#.
# ....###..#
# ..#.#..#.#
# #..#.#.###
# .##...##.#
# .....#.#..
# """

asteroids = np.array([[c == '#' for c in line] for line in asteroids.splitlines()])

In [3]:
h, w = asteroids.shape
X, Y = np.mgrid[:h, :w]
visible = np.zeros((h, w), dtype=int)

for (x, y), ast in np.ndenumerate(asteroids):
    if ast:
        mask = asteroids.copy()
        mask[x, y] = False

        distX = -(X - x)
        distY = Y - y

        ang = np.around(np.angle(distX+distY*1j, deg=True), 8)
        ang[ang < 0] += 360
        ang[~mask] = np.nan

        visible[x, y] = len(np.unique(ang[mask]))
display(np.max(visible), np.argwhere(visible == np.max(visible)))

210

array([[13, 11]])

In [4]:
x, y = 11, 13

def new_map(asteroids, x, y):
    mask = asteroids.copy()
    mask[x, y] = False

    h, w = asteroids.shape
    X, Y = np.mgrid[:h, :w]
    distX = -(X - x)
    distY = Y - y

    ang = np.around(np.angle(distX+distY*1j, deg=True), 8)
    ang[ang < 0] += 360
    ang[~mask] = np.nan

    abs = np.around(np.abs(distX+distY*1j), 8)
    abs[~mask] = np.nan

    return ang, abs

it = 0
lasered = 0

while True:
    it += 1
    print("Iteration", it)
    ang, abs = new_map(asteroids, x, y)

    smallest_angle = -1
    while True:
        smallest_angle   = np.nanmin(ang[ang > smallest_angle])
        astroids_on_line = np.where(ang == smallest_angle)
        nearest_on_line  = np.argmin(abs[astroids_on_line])
        nearest_x        = astroids_on_line[0][nearest_on_line]
        nearest_y        = astroids_on_line[1][nearest_on_line]

        asteroids[nearest_x, nearest_y] = False
        lasered += 1
        print(lasered, nearest_x, nearest_y)
        # print(smallest_angle)

        if np.all(ang[asteroids] <= smallest_angle):
            break
    if np.count_nonzero(asteroids) <= 1:
        break

Iteration 1
1 10 13
2 0 14
3 1 14
4 2 14
5 3 14
6 4 14
7 5 14
8 0 15
9 6 14
10 2 15
11 7 14
12 0 16
13 4 15
14 8 14
15 0 17
16 3 16
17 6 15
18 4 16
19 2 17
20 0 18
21 9 14
22 0 19
23 6 16
24 3 18
25 8 15
26 4 18
27 7 16
28 6 17
29 5 18
30 4 19
31 10 14
32 6 19
33 7 18
34 8 17
35 9 16
36 8 18
37 10 15
38 10 16
39 10 17
40 10 18
41 10 19
42 11 15
43 12 19
44 12 18
45 12 17
46 12 15
47 14 18
48 13 16
49 14 17
50 15 18
51 16 19
52 12 14
53 18 19
54 17 18
55 16 17
56 15 16
57 18 18
58 14 15
59 19 18
60 16 16
61 18 17
62 13 14
63 16 15
64 14 14
65 18 15
66 15 14
67 17 14
68 18 14
69 12 13
70 19 12
71 18 12
72 17 12
73 16 12
74 15 12
75 17 11
76 19 10
77 16 11
78 18 10
79 13 12
80 18 9
81 16 10
82 19 8
83 14 11
84 18 8
85 15 10
86 17 8
87 18 7
88 12 12
89 19 4
90 17 6
91 16 7
92 15 8
93 18 4
94 14 9
95 19 2
96 13 10
97 18 2
98 16 5
99 19 0
100 14 8
101 15 6
102 16 4
103 18 0
104 12 11
105 15 4
106 14 6
107 13 8
108 14 5
109 15 2
110 12 10
111 14 3
112 14 2
113 13 5
114 14 0
115 12 8
116 13 1
